In [7]:
!pip install python_deriv_api --user

  Using cached python_deriv_api-0.1.1-py3-none-any.whl (40 kB)
  Using cached Rx-3.1.1-py3-none-any.whl (197 kB)


You should consider upgrading via the 'C:\Users\nikhi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [33]:
from deriv_api import DerivAPI
import pandas as pd

In [3]:
api_token = 'O4WJddI6GWcNskz'
app_id = 31998

In [4]:
api = DerivAPI(app_id=app_id)

In [5]:
authorize = await api.authorize(api_token)
print(authorize)

{'authorize': {'account_list': [{'account_type': 'trading', 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'loginid': 'CR3963482', 'trading': {}}, {'account_type': 'trading', 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'loginid': 'VRTC6109201', 'trading': {}}], 'balance': 10053.85, 'country': 'in', 'currency': 'USD', 'email': 'ravitejavenkat.k5@gmail.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'local_currencies': {'INR': {'fractional_digits': 2}}, 'loginid': 'VRTC6109201', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin', 'trading_information'], 'trading': {}, 'upgradeable_landing_companies': ['svg'], 'user_id': 10222109}, 'echo_req': {'authorize': '<not shown>', 'req_id': 1}, 'msg_type': 'authorize', 'req_id': 1}


In [6]:
account = await api.balance()
print(account)

{'balance': {'balance': 10053.85, 'currency': 'USD', 'loginid': 'VRTC6109201'}, 'echo_req': {'balance': 1, 'req_id': 2}, 'msg_type': 'balance', 'req_id': 2}


In [7]:
active_symbols = await api.active_symbols({"active_symbols": "full"})
print(active_symbols)

{'active_symbols': [{'allow_forward_starting': 0, 'delay_amount': 0, 'display_name': 'AUD Basket', 'exchange_is_open': 1, 'exchange_name': 'FOREX', 'intraday_interval_minutes': 5, 'is_trading_suspended': 0, 'market': 'basket_index', 'market_display_name': 'Basket Indices', 'pip': 0.001, 'quoted_currency_symbol': 'WLDAUD', 'spot': 926.378, 'spot_age': '1', 'spot_time': '1654029330', 'submarket': 'forex_basket', 'submarket_display_name': 'Forex Basket', 'symbol': 'WLDAUD', 'symbol_type': 'forex_basket'}, {'allow_forward_starting': 1, 'delay_amount': 0, 'display_name': 'AUD/CAD', 'exchange_is_open': 1, 'exchange_name': 'FOREX', 'intraday_interval_minutes': 5, 'is_trading_suspended': 0, 'market': 'forex', 'market_display_name': 'Forex', 'pip': 1e-05, 'quoted_currency_symbol': 'CAD', 'spot': 0.90796, 'spot_age': '1', 'spot_time': '1654029330', 'submarket': 'minor_pairs', 'submarket_display_name': 'Minor Pairs', 'symbol': 'frxAUDCAD', 'symbol_type': 'forex'}, {'allow_forward_starting': 1, 'd

In [24]:
symbol = 'GBP/USD'
our_types = ['ONETOUCH', 'NOTOUCH']

In [18]:
deriv_symbol = [a['symbol'] for a in active_symbols['active_symbols'] if a['display_name'] == symbol][0]
deriv_symbol

'frxGBPUSD'

In [25]:
contract_types = await api.contracts_for({
    "contracts_for": deriv_symbol,
    "currency": "USD",
    "landing_company": "svg",
    "product_type": "basic"
})

print([(a['contract_display'], a['min_contract_duration']) for a in contract_types['contracts_for']['available'] if
       a['contract_type'] in our_types])

[('Touches', '1d'), ('Does Not Touch', '1d')]


In [65]:
proposal = await api.proposal({"proposal": 1, "amount": 100, "barrier": "1.2652", "basis": "stake",
                               "contract_type": our_types[0], "currency": "USD", "duration": 1, "duration_unit": "d",
                               "symbol": deriv_symbol
                               })
print(proposal)

{'echo_req': {'req_id': 28, 'subscribe': 1, 'ticks': 'R_50'}, 'msg_type': 'tick', 'req_id': 28, 'subscription': {'id': '0d77256d-964b-78c7-5cd8-96f0d13f134d'}, 'tick': {'ask': 190.6212, 'bid': 190.6012, 'epoch': 1654033172, 'id': '0d77256d-964b-78c7-5cd8-96f0d13f134d', 'pip_size': 4, 'quote': 190.6112, 'symbol': 'R_50'}}
{'echo_req': {'amount': 100, 'barrier': '1.2652', 'basis': 'stake', 'contract_type': 'ONETOUCH', 'currency': 'USD', 'duration': 1, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'req_id': 35, 'symbol': 'frxGBPUSD'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 100, 'date_expiry': 1654127999, 'date_start': 1654033172, 'display_value': '100.00', 'id': 'ed85b38d-e4cc-d287-6b29-f61969c6e53a', 'longcode': 'Win payout if GBP/USD touches 1.26520 through close on 2022-06-01.', 'payout': 146.06, 'spot': 1.26014, 'spot_time': 1654033171}, 'req_id': 35}


In [64]:
1.2602 * 1.004

1.2652408

In [30]:
proposal_id = proposal['proposal']['id']
buy = await api.buy({"buy": proposal_id, "price": 100})
print(buy)

{'buy': {'balance_after': 9953.85, 'buy_price': 100, 'contract_id': 171635196208, 'longcode': 'Win payout if GBP/USD touches 1.20000 through close on 2022-06-01.', 'payout': 2000, 'purchase_time': 1654031228, 'shortcode': 'ONETOUCH_FRXGBPUSD_2000.00_1654031228_1654127999_1200000_0', 'start_time': 1654031228, 'transaction_id': 342584007288}, 'echo_req': {'buy': '9ce0ce27-dc38-4100-23ba-10f147e8d081', 'price': 100, 'req_id': 16}, 'msg_type': 'buy', 'req_id': 16}


In [31]:
contract_id = buy['buy']['contract_id']
poc = await api.proposal_open_contract(
    {"proposal_open_contract": 1, "contract_id": contract_id})
print(poc)

{'echo_req': {'contract_id': 171635196208, 'proposal_open_contract': 1, 'req_id': 17}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 126086828, 'barrier': '1.20000', 'barrier_count': 1, 'bid_price': 0, 'buy_price': 100, 'contract_id': 171635196208, 'contract_type': 'ONETOUCH', 'currency': 'USD', 'current_spot': 1.26023, 'current_spot_display_value': '1.26023', 'current_spot_time': 1654031291, 'date_expiry': 1654127999, 'date_settlement': 1654128000, 'date_start': 1654031228, 'display_name': 'GBP/USD', 'entry_spot': 1.26018, 'entry_spot_display_value': '1.26018', 'entry_tick': 1.26018, 'entry_tick_display_value': '1.26018', 'entry_tick_time': 1654031229, 'expiry_time': 1654127999, 'is_expired': 0, 'is_forward_starting': 0, 'is_intraday': 0, 'is_path_dependent': 1, 'is_settleable': 0, 'is_sold': 0, 'is_valid_to_cancel': 0, 'is_valid_to_sell': 0, 'longcode': 'Win payout if GBP/USD touches 1.20000 through close on 2022-06-01.', 'payout': 2000, 'profit': -10

In [32]:
profit_table = await api.profit_table({"profit_table": 1, "description": 1, "sort": "ASC"})
print(profit_table)

{'echo_req': {'description': 1, 'profit_table': 1, 'req_id': 18, 'sort': 'ASC'}, 'msg_type': 'profit_table', 'profit_table': {'count': 23, 'transactions': [{'app_id': 16929, 'buy_price': 10, 'contract_id': 171432001028, 'duration_type': 'ticks', 'longcode': 'Win payout if Volatility 100 (1s) Index after 5 ticks is strictly lower than entry spot.', 'payout': 19.53, 'purchase_time': 1653917041, 'sell_price': 0, 'sell_time': 1653917047, 'shortcode': 'PUT_1HZ100V_19.53_1653917041_5T_S0P_0', 'transaction_id': 342177783988}, {'app_id': 16929, 'buy_price': 10, 'contract_id': 171432048748, 'duration_type': 'ticks', 'longcode': 'Win payout if Volatility 100 (1s) Index touches entry spot plus 1.22 through 5 ticks after first tick.', 'payout': 33.16, 'purchase_time': 1653917066, 'sell_price': 0, 'sell_time': 1653917073, 'shortcode': 'ONETOUCH_1HZ100V_33.16_1653917066_5T_S122P_0', 'transaction_id': 342177878888}, {'app_id': 16929, 'buy_price': 10, 'contract_id': 171432108508, 'duration_type': 'min

In [34]:
df = pd.DataFrame(profit_table['profit_table']['transactions'])
df.head()

,app_id,buy_price,contract_id,duration_type,longcode,payout,purchase_time,sell_price,sell_time,shortcode,transaction_id
0,16929,10,171432001028,ticks,Win payout if Volatility 100 (1s) Index after ...,19.53,1653917041,0.00,1653917047,PUT_1HZ100V_19.53_1653917041_5T_S0P_0,342177783988
1,16929,10,171432048748,ticks,Win payout if Volatility 100 (1s) Index touche...,33.16,1653917066,0.00,1653917073,ONETOUCH_1HZ100V_33.16_1653917066_5T_S122P_0,342177878888
2,16929,10,171432108508,minutes,Win payout if Volatility 100 (1s) Index touche...,14.29,1653917096,14.29,1653917336,ONETOUCH_1HZ100V_14.29_1653917096_1653917696_S...,342177997028
3,16929,10,171432355368,minutes,Win payout if Volatility 100 (1s) Index touche...,29.10,1653917216,29.10,1653917570,ONETOUCH_1HZ100V_29.10_1653917216_1653917816_S...,342178490508
4,16929,10,171434171808,days,Win payout if Volatility 100 (1s) Index touche...,400.00,1653918090,0.00,1653918101,ONETOUCH_1HZ100V_400.00_1653918090_1654004490_...,342182123148


In [36]:
statement = await api.statement({"statement": 1, "description": 1, "limit": 100, "offset": 25})
print(statement)

{'echo_req': {'description': 1, 'limit': 100, 'offset': 25, 'req_id': 19, 'statement': 1}, 'msg_type': 'statement', 'req_id': 19, 'statement': {'count': 24, 'transactions': [{'action_type': 'sell', 'amount': 0, 'app_id': 2, 'balance_after': 9981.03, 'contract_id': 171436584368, 'longcode': 'Win payout if Volatility 100 (1s) Index touches entry spot plus 6.04 through 10 minutes after contract start time.', 'payout': 14.28, 'purchase_time': 1653919235, 'reference_id': 342186946468, 'shortcode': 'ONETOUCH_1HZ100V_14.28_1653919235_1653919835_S604P_0', 'transaction_id': 342189443888, 'transaction_time': 1653919836}, {'action_type': 'buy', 'amount': -10, 'app_id': 16929, 'balance_after': 9981.03, 'contract_id': 171436584368, 'longcode': 'Win payout if Volatility 100 (1s) Index touches entry spot plus 6.04 through 10 minutes after contract start time.', 'payout': 14.28, 'reference_id': None, 'shortcode': 'ONETOUCH_1HZ100V_14.28_1653919235_1653919835_S604P_0', 'transaction_id': 342186946468, '

In [37]:
df = pd.DataFrame(statement['statement']['transactions'])
df.head()

,action_type,amount,app_id,balance_after,contract_id,longcode,payout,purchase_time,reference_id,shortcode,transaction_id,transaction_time
0,sell,0.00,2,9981.03,1.714366e+11,Win payout if Volatility 100 (1s) Index touche...,14.28,1.653919e+09,3.421869e+11,ONETOUCH_1HZ100V_14.28_1653919235_1653919835_S...,342189443888,1653919836
1,buy,-10.00,16929,9981.03,1.714366e+11,Win payout if Volatility 100 (1s) Index touche...,14.28,NaN,NaN,ONETOUCH_1HZ100V_14.28_1653919235_1653919835_S...,342186946468,1653919235
2,sell,14.30,2,9991.03,1.714362e+11,Win payout if Volatility 100 (1s) Index touche...,14.30,1.653919e+09,3.421861e+11,ONETOUCH_1HZ100V_14.30_1653919027_1653919627_S...,342186772688,1653919194
3,sell,10.69,16929,9976.73,1.714347e+11,Win payout if Volatility 100 (1s) Index touche...,32.21,1.653918e+09,3.421832e+11,ONETOUCH_1HZ100V_32.21_1653918355_1654004755_S...,342186241828,1653919063
4,sell,9.61,16929,9966.04,1.714358e+11,Win payout if Volatility 100 (1s) Index touche...,21.77,1.653919e+09,3.421854e+11,ONETOUCH_1HZ100V_21.77_1653918874_1654041599_3...,342186218448,1653919058


In [38]:
df

,action_type,amount,app_id,balance_after,contract_id,longcode,payout,purchase_time,reference_id,shortcode,transaction_id,transaction_time
0,sell,0.00,2,9981.03,1.714366e+11,Win payout if Volatility 100 (1s) Index touche...,14.28,1.653919e+09,3.421869e+11,ONETOUCH_1HZ100V_14.28_1653919235_1653919835_S...,342189443888,1653919836
1,buy,-10.00,16929,9981.03,1.714366e+11,Win payout if Volatility 100 (1s) Index touche...,14.28,NaN,NaN,ONETOUCH_1HZ100V_14.28_1653919235_1653919835_S...,342186946468,1653919235
2,sell,14.30,2,9991.03,1.714362e+11,Win payout if Volatility 100 (1s) Index touche...,14.30,1.653919e+09,3.421861e+11,ONETOUCH_1HZ100V_14.30_1653919027_1653919627_S...,342186772688,1653919194
3,sell,10.69,16929,9976.73,1.714347e+11,Win payout if Volatility 100 (1s) Index touche...,32.21,1.653918e+09,3.421832e+11,ONETOUCH_1HZ100V_32.21_1653918355_1654004755_S...,342186241828,1653919063
4,sell,9.61,16929,9966.04,1.714358e+11,Win payout if Volatility 100 (1s) Index touche...,21.77,1.653919e+09,3.421854e+11,ONETOUCH_1HZ100V_21.77_1653918874_1654041599_3...,342186218448,1653919058
5,buy,-10.00,16929,9956.43,1.714362e+11,Win payout if Volatility 100 (1s) Index touche...,14.30,NaN,NaN,ONETOUCH_1HZ100V_14.30_1653919027_1653919627_S...,342186093608,1653919027
6,sell,14.30,2,9966.43,1.714360e+11,Win payout if Volatility 100 (1s) Index touche...,14.30,1.653919e+09,3.421858e+11,ONETOUCH_1HZ100V_14.30_1653918949_1653919549_S...,342185990408,1653919004
7,buy,-10.00,16929,9952.13,1.714360e+11,Win payout if Volatility 100 (1s) Index touche...,14.30,NaN,NaN,ONETOUCH_1HZ100V_14.30_1653918949_1653919549_S...,342185761108,1653918949
8,buy,-10.00,16929,9962.13,1.714358e+11,Win payout if Volatility 100 (1s) Index touche...,21.77,NaN,NaN,ONETOUCH_1HZ100V_21.77_1653918874_1654041599_3...,342185445668,1653918874
9,sell,8.74,16929,9972.13,1.714350e+11,Win payout if AUD/JPY touches 92.276 through c...,18.49,1.653918e+09,3.421838e+11,ONETOUCH_FRXAUDJPY_18.49_1653918479_1654041599...,342185206768,1653918817


In [52]:
source_poc = await api.subscribe({"proposal_open_contract": 1, "contract_id": contract_id})
rxpy = source_poc.subscribe(lambda poc: print(poc))

In [ ]:
# Monitor the is_sold attribute of the poc. Then, once the contract is finished, you'll know if it's won or not.

In [ ]:
# Todo: Get live ticker price of a symbol.
# Get historical data of a symbol.
# Utilize the websockets somehow from the Quickstart guide.
# https://api.deriv.com/api-explorer/#ticks_history

In [60]:
rxpy.dispose()

In [1]:
from dateutil import parser

In [13]:
parser.parse('2022-05-31').timestamp()

1653951600.0

In [12]:
from get_data import get_deriv_data

ModuleNotFoundError: No module named 'deriv_api'

In [11]:
import sys
print(sys.executable)

C:\Users\nikhi\AppData\Local\Programs\Python\Python310\python.exe
